# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle
import wget, os.path, zipfile, glob #get dataset
import numpy as np
import random, time
import csv 
import tensorflow as tf

def downloadDataset(download_path):
    filename = './dataset/traffic-signs-data.zip'

    url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip'
    if not os.path.exists(filename):
        wget.download(url, filename)
    
    if os.path.exists(filename):
        zip_ref = zipfile.ZipFile(filename, 'r')
        zip_ref.extractall(download_path)
        zip_ref.close()
    
    files = glob.glob(download_path+'*.p')
    return len(files) >= 3

def loadLabelDescription(filename):
    label_text = {}
    with open(filename, 'r') as fp:
        reader = csv.reader(fp)
        for line in reader:
            label_text[int(line[0])]=line[1]
    res = [None]*len(label_text)
    for key in label_text:
        res[key]=label_text[key]
    return res

def loadDataset(path, name):
    file = path + name + '.p'
    if os.path.exists(file):
        with open(file, mode='rb') as f:
            dataset = pickle.load(f)
        return dataset 

    
class_texts = loadLabelDescription('signnames.csv') # print(label_text)
dataset_path = './dataset/traffic-signs-data/'
downloadResult = downloadDataset(dataset_path)
assert downloadResult 

train = loadDataset(dataset_path, 'train')
valid = loadDataset(dataset_path, 'valid')
test = loadDataset(dataset_path, 'test')

X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']


---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = len(X_train)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = str(len(X_train[0]))+'x'+str(len(X_train[0][0]))

# TODO: How many unique classes/labels there are in the dataset.
class_labels = np.sort(np.unique(y_train))
n_classes = len(class_labels)
class_count = [None]*n_classes
class_examples = [None]*n_classes
class_share = [float]*n_classes


print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

for label in class_labels:
    class_examples[label] = X_train[y_train == label]
    class_count[label] = len(class_examples[label])
    class_share[label] = round(float(class_count[label]*100)/n_train,1)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
import random,math
# Visualizations will be shown in the notebook.
%matplotlib inline

cols = 7
lines = [None]*(math.ceil(n_classes / cols) -1)
line = [None]*cols
for label in class_labels:
    row, draw = divmod(label, cols)
    offset = row * cols
    rand_idx = math.floor(len(class_examples[label])*random.random())
    line[draw] = class_examples[label][rand_idx]
    if draw == cols-1:
        lines[row] = np.concatenate(line, axis=1)

lines = np.concatenate(np.array(lines), axis=0)
fig, ax = plt.subplots(figsize=(40, 20))
ax.imshow(lines, interpolation="bicubic")
ax.axis('off')  # clear x- and y-axes
plt.show()


In [ ]:
ind = np.arange(n_classes)  # the x locations for the groups
width = 0.35       # the width of the bars
fig, ax= plt.subplots(figsize=(20,10))
rects1 = ax.barh(ind, class_count, width, color='r')
ax.set_xlabel('Nr.')
ax.set_title('Number of example per class')
ax.set_yticks(ind)
ax.set_yticklabels(class_texts)
plt.show()

In [ ]:
for label in class_labels:
    print("| %d \t| %.1f%% \t| %s \t" %(class_count[label],class_share[label],class_texts[label]) )

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:

### Preprocess the data here. Preprocessing steps could include normalization, converting to grayscale, etc.
### Feel free to use as many code cells as needed.
from sklearn.utils import shuffle
import cv2
import os

def normalizeImage(image):
    #Homemade version of https://www.tensorflow.org/api_docs/python/tf/image/per_image_standardization
    #TF version is messy to use (no pickly so -> eval, sessions, etc)
    #print('Before',image[0][0][0])
    count = np.prod(image.shape)
    mean = np.mean(image)
    std = np.std(image)
    adj_std = max(std, 1.0/np.sqrt(count))
    image = (image - mean) / adj_std
    #print('After',image[0][0][0])
    return image


def histogramEqualization(image):
    #
    ycrcb=cv2.cvtColor(image,cv2.COLOR_BGR2YCR_CB)
    channels=cv2.split(ycrcb)
    print(len(channels))
    cv2.equalizeHist(channels[0],channels[0])
    cv2.merge(channels,ycrcb)
    cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2BGR,image)
    return image

def processImage(image):
    #image = histogramEqualization(image) #maybe yes, maybe no
    image = normalizeImage(image)
    return image

print("Normalizing training set")
X_train = [processImage(image) for image in X_train]
print("Normalizing validation set")
X_valid = [processImage(image) for image in X_valid]
print("Normalizing test set")
X_test = [processImage(image) for image in X_test]




print("Shuffle train set")
X_train, y_train = shuffle(X_train, y_train)

### Model Architecture

In [ ]:
# network building functions
from tensorflow.contrib.layers import flatten

def layerShapeSize(layer):
    shape = layer.get_shape()
    size = 1
    for dim in shape:
        size *= int(dim) if str(dim) != '?' else 1
    return shape, size

def convLayer(input_layer, params, defaults):
    input_depth = int(input_layer.get_shape()[3])
    
    name         = params['name']
    ksize        = params['ksize']   if 'ksize'   in params else 3
    output_depth = params['depth']   if 'depth'   in params else input_depth
    padding      = params['padding'] if 'padding' in params else 'SAME'
    stride       = params['stride']  if 'stride'  in params else 1
    
    dropout      = params['dropout'] if 'dropout' in params else defaults['dropout'] if 'dropout' else 1.0 #1.0 disable by default
    mu           = params['mu']      if 'mu'      in params else defaults['mu']      if 'mu'      else 0
    sigma        = params['sigma']   if 'sigma'   in params else defaults['sigma']   if 'sigma'   else 0.1
    
    name_W = name + '_W'
    name_b = name + '_b'
    with tf.variable_scope(name):
        W = tf.get_variable( name_W, (ksize, ksize, input_depth, output_depth), initializer=tf.truncated_normal_initializer(mean=mu, stddev=sigma) )
        b = tf.get_variable( name_b, (output_depth), initializer=tf.zero_initializer() )
    #W = tf.Variable(tf.truncated_normal(shape=(ksize, ksize, input_depth, output_depth), mean = mu, stddev = sigma), name = name_W)
    #b = tf.Variable(tf.zeros(output_depth),name = name_b)
    conv = tf.nn.conv2d(input_layer, W, strides=[1, stride, stride, 1], padding=padding) + b
    conv = tf.nn.relu(conv)
    
    layerInfo = {
        'variables':{name_W:W,name_b:b},
        'placeholders':{},
    }
    
    if dropout < 1:
        name_keep_prob = name+'_keep_prob'
        keep_prob = tf.placeholder(tf.float32,shape=(),name=name_keep_prob)
        layerInfo['placeholders'][name_keep_prob] = keep_prob
        conv = tf.nn.dropout(conv,dropout)

    return conv, params, layerInfo

def poolLayer(input_layer, params, defaults):
    ksize   = params['ksize']   if 'ksize'   in params else 2
    stride  = params['stride']  if 'stride'  in params else ksize
    padding = params['padding'] if 'padding' in params else 'SAME'
    pool = tf.nn.max_pool(input_layer, ksize=[1, ksize, ksize, 1], strides=[1, stride, stride, 1], padding=padding)
    layerInfo = {'variables':{}, 'placeholders':{}}
    return pool, params, layerInfo

def poolDepthLayer(input_layer, params, defaults):
    ksize   = params['ksize']   if 'ksize'   in params else 2
    stride  = params['stride']  if 'stride'  in params else ksize
    padding = params['padding'] if 'padding' in params else 'VALID'
    pool = tf.nn.max_pool(input_layer, ksize=[1, 1, 1, stride], strides=[1, 1, 1, ksize], padding=padding)
    layerInfo = {'variables':{}, 'placeholders':{}}
    return pool, params, layerInfo


def fcLayer(input_layer, params, defaults):
    input_size = int(input_layer.get_shape()[1])
    
    name         = params['name']
    size         = params['size']         if 'size' in params else input_size
    activation   = params['activation']   if 'activation' in params else True
    
    dropout      = params['dropout'] if 'dropout' in params else defaults['dropout'] if 'dropout' else 1.0 #1.0 disable by default
    mu           = params['mu']      if 'mu' in params else defaults['mu'] if 'mu' else 0
    sigma        = params['sigma']   if 'sigma' in params else defaults['sigma'] if 'sigma' else 0.1
    
    name_W = name + '_W'
    name_b = name + '_b'
    with tf.variable_scope(name, reuse=None):
        W = tf.get_variable( name_W, (input_size, size), tf.truncated_normal_initializer(mean=mu, stddev=sigma )        )
        b = tf.get_variable( name_b, (output_depth), tf.zero_initializer() )

    

    #W  = tf.Variable(tf.truncated_normal(shape=(input_size, size), mean = mu, stddev = sigma),name=name_W)
    #b  = tf.Variable(tf.zeros(size),name=name_b)
    fc = tf.matmul(input_layer, W) + b
    layerInfo = {'variables':{name_W:W,name_b:b}, 'placeholders':{}}
    if activation:
        fc    = tf.nn.relu(fc)
        if dropout < 1:
            name_keep_prob = name+'_keep_prob'
            keep_prob = tf.placeholder(tf.float32,shape=(),name=name_keep_prob)
            layerInfo['placeholders'][name_keep_prob]=keep_prob
            fc = tf.nn.dropout(fc,dropout)
    
    return fc, params, layerInfo

def flatLayer(input_layer,params, defaults):
    flat = flatten(input_layer)
    layerInfo = {'variables':{}, 'placeholders':{}}
    return flat, params, layerInfo



def BuildNet(x,hyperparams):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    network_size = 0
    layer_count = 0
    top_layer = x
    modelParam = {}
    modelInfo = {}

    
    name_str = 'input'.ljust(10)
    shape, layer_size = layerShapeSize(top_layer)
    print(name_str,':\t',shape,'\t', layer_size)
    
    layer_builder = {
        'conv'       : convLayer,
        'pool'       : poolLayer,
        'pool-depth' : poolDepthLayer,
        'flat'       : flatLayer,
        'fc'         : fcLayer,
    }
    
    defaults = hyperparams['defaults']
    
    with tf.variable_scope("layer", reuse=None) as scope:
        for layerParams in hyperparams['model']:
            kind = layerParams['kind']
            layerParams['name'] = layerParams['name'] if 'name' in layerParams else "%d_%s" % (layer_count, kind)
            name = layerParams['name']

            if kind not in layer_builder: raise ValueError('Unknown Layer kind: ' + kind)
            top_layer, layerParams, layerInfo = layer_builder[kind](top_layer, layerParams, defaults) 
            layerInfo['layer'] = top_layer
            modelParam[name] = layerParams
            modelInfo[name] = layerInfo

            name_str = name.ljust(10)
            shape, layer_size = layerShapeSize(top_layer)
            print(name_str,':\t',shape,'\t', layer_size)
            network_size += layer_size

            layer_count += 1
    
    return top_layer, modelParam, modelInfo


def evaluate(batch_size, accuracy_operation, x, y, X_data, y_data, feed_dict={}):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, batch_size):
        batch_x, batch_y = X_data[offset:offset+batch_size], y_data[offset:offset+batch_size]
        feed_dict.update({x: batch_x, y: batch_y})
        accuracy = sess.run(accuracy_operation, feed_dict=feed_dict)
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
import pprint, datetime, time
from htmltoolkit import HtmlToolkit 



def runTraining(hyperparams,train,valid,test):
    #output tools
    pp = pprint.PrettyPrinter()
    htk = HtmlToolkit()
    js = htk.js
    jQuery = htk.jQuery
    
    #unpack dataset
    X_train, y_train = train
    X_valid, y_valid = valid
    X_test, y_test = test
    
    #unpack global parameters
    n_classes = hyperparams['n_classes']
    epochs = hyperparams['epochs']
    batch_size = hyperparams['batch_size']
    rate = hyperparams['rate']
    rate_step = hyperparams['rate_step']
    rate_change_limit = hyperparams['rate_change_limit'] 
    
    #define model name and storage folder
    basedir = './checkpoints/'
    filename = datetime.datetime.now().strftime('%Y%m%d-%H%M%S') #save the current model into todays run
    modeldir = basedir + filename+'/'
    os.mkdir(modeldir)
    info_name = modeldir + 'info-' +filename
    model_name = modeldir + 'model-' +filename
    
    #define inputs
    learning_rate = tf.placeholder(tf.float32,shape=(),name="learning_rate")
    x = tf.placeholder(tf.float32, (None, 32, 32, 3))
    y = tf.placeholder(tf.int32, (None))
    one_hot_y = tf.one_hot(y, n_classes)
    
    #build net
    logits, modelParams, modelInfo = BuildNet(x,hyperparams)
    
    #define operations
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)
    loss_operation = tf.reduce_mean(cross_entropy)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    training_operation = optimizer.minimize(loss_operation)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
    accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    #build feed_dict for tran and evauate (*_keep_prob)
    feed_dict_train = {}
    feed_dict_evaluate = {}
    for layerName in modelInfo:
        layerInfo = modelInfo[layerName]
        for name in layerInfo['placeholders']:
            if name.endswith('_keep_prob'):
                #print(layerName,name)
                plc = layerInfo['placeholders'][name]
                layerParam = modelParams[layerName]
                keep_prob = layerParam['dropout'] if 'dropout' in layerParam else hyperparams['defaults']['dropout']
                feed_dict_train[plc] = float(keep_prob)
                feed_dict_evaluate[plc] = 1.0
            else:
                print('Trovato placeholder non gestito :(',name)
    feed_dict_train[learning_rate] = rate #add learning rate to train feed_dict only
    
    #print(feed_dict_train)
    #print(feed_dict_evaluate)
    
    
    #save current configuration before running, just in case...
    utils.saveModelInfo(info_name,0,hyperparams,0,0)
    saver = tf.train.Saver()
    
    start = time.time()
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())
        num_examples = len(X_train)
        time_history = []
        accuracy_history = []
        accuracy_avg5_history = []
        accuracy_diff_history = []
        accuracy_rate_history = []
        print()
        print('Current configuration:')
        pp.pprint(hyperparams)
        print()
        print("Training...")
        
        labels = [[
            'Epoch',
            'Rate',
            #'Acc Train',
            'Acc. Valid',
            'Acc. Avg Last 5',
            'Δ Acc. ',
            'Δ Total',
            'Δ Last 5',
            'Avg Δ Last 5 ',
            'Train',
            'Save',
            #'GPU RAM'
        ]]
        table = htk.htmlTable(labels,{'id':'epoch-table'})
        htk.printHTML(table)
        
        last_epoch_changed = 0
        rate_changed = ' '
        for i in range(epochs):
            angle = 0
            start_epoch = time.time()
            X_train, y_train = shuffle(X_train, y_train)
            for offset in range(0, num_examples, batch_size):
                end = offset + batch_size
                batch_x, batch_y = X_train[offset:end], y_train[offset:end]
                feed_dict_train.update({x: batch_x, y: batch_y})
                sess.run(training_operation, feed_dict=feed_dict_train)

            memTotal,memFree,memFull=[0,0,0] #memInfo()
            #print("Total:{:d}, Free: {:d}, Full: {:d}".format(memTotal,memFree,memFull))


            train_accuracy = 0# evaluate(X_train, y_train, keep_prob_list)
            validation_accuracy = evaluate(batch_size, accuracy_operation, x, y, X_valid, y_valid, feed_dict_evaluate)
            #print(evaluate(X_valid, y_valid))

            delta_epoch = time.time()-start_epoch
            delta_all = time.time()-start

            time_history.append(delta_epoch)
            accuracy_history.append(validation_accuracy)
 
            accuracy_diff = accuracy_history[i]-accuracy_history[i-1] if i > 0 else 0
            accuracy_rate = (accuracy_diff*100)/accuracy_history[i]

            accuracy_diff_history.append(accuracy_diff)
            accuracy_rate_history.append(accuracy_rate)

            last_count = 5
            lastN = []
            if i > 0:
                capped = accuracy_rate_history[1:]
                lastN = capped[-min(i,5):]

            lastN = lastN if len(lastN)>0 else [0]
            avgN = np.average(lastN)

            avgN_acc = np.average(accuracy_history[-5:])
            accuracy_avg5_history.append(avgN_acc)

            delta_save = '-'
            save_every = 4
            if (i+1) % save_every == 1 or i == epochs-1: # save the first run, the last and every 'save_every'
                start_save = time.time()
                utils.saveModelInfo(info_name,i,hyperparams,time_history,accuracy_history)
                saver.save(sess, model_name)        
                delta_save = time.time() - start_save
                delta_save = str(round(delta_save))+'s'

            new_rate = rate
            if i > last_epoch_changed+last_count:
                if avgN < rate_change_limit:
                    new_rate *= rate_step
                    feed_dict_train[learning_rate] = rate
                    rate_changed = '▼' #'☹'
                    last_epoch_changed = i
                else:
                    rate_changed = ' '
            else:
                rate_changed = '🔒'
                
            lineValues =(
                "{:d}".format(i+1),
                "10^{:d}{}".format( int(np.log10(rate)), rate_changed ),
                #"{:.3f}".format( train_accuracy ),
                "{:.3f}".format( validation_accuracy ),
                "{:.3f}".format( avgN_acc ),
                "{:+.3f}".format( accuracy_diff ),
                "{:+.3f}".format( np.sum(accuracy_diff_history) ),
                "{:+.3f}%".format( accuracy_rate ),
                "{:+.3f}%".format( avgN ),
                "{:.1f}s".format( delta_epoch ),
                "{}".format( delta_save ),
                #'{:d} ({:d}%)'.format( memFull, ((memFull*100)/memTotal) if memTotal >0 else 0 )
            ) 
            row = htk.htmlTableRow(lineValues,{'id':'epoch_'+str(i+1)})
            jQuery("#epoch-table").append(row)

            

            rate = new_rate
            

        print("trainig finished")
        train_accuracy = validation_accuracy = evaluate(batch_size, accuracy_operation, x, y, X_train, y_train, feed_dict_evaluate)
        validation_accuracy = evaluate(batch_size, accuracy_operation, x, y, X_valid, y_valid, feed_dict_evaluate)
        test_accuracy = evaluate(batch_size, accuracy_operation, x, y, X_test, y_test, feed_dict_evaluate)
        print("Train: {}\nValidation: {}\nTest: {}".format(train_accuracy,validation_accuracy,test_accuracy))
        return sess, [logits, modelParams, modelInfo], [correct_prediction, accuracy_operation]

    
    
    

In [ ]:
# parameters section

hyperparams ={
    "epochs": 40,
    "batch_size": 1,
    
    'rate': 0.001,
    'rate_step': 0.1,
    'rate_change_limit': 0.001, #currently last5 avg5
    
    "n_classes": n_classes,
    "defaults":{
        "mu": 0,
        "sigma": 0.1,
        "dropout": 1.0,
    },

    "model": [
        { "kind": "conv", "depth": 10 },
        { "kind": "conv", "depth": 20},
        { "kind": "pool", "padding": "VALID" },
        { "kind": "conv", "depth": 40, "padding": "VALID" },
        { "kind": "conv", "depth": 60, "padding": "VALID" },
        { "kind": "pool", "padding": "VALID" },
        { "kind": "conv", "depth": 80, "padding": "VALID" },
        { "kind": "flat" }, 
        { "kind": "fc", "size": 600, "dropout": 0.5}, 
        { "kind": "fc", "size": 400},
        { "kind": "fc", "size": 200}, 
        { "kind": "fc", "size": 100, "dropout": 0.5}, 
        { "kind": "fc", "size": n_classes, "activation": False}
    ]
} 

sess, model, operations = runTraining(
    hyperparams,
    ( X_train, y_train ),
    ( X_valid, y_valid ),
    ( X_test,  y_test )
)




In [ ]:
#print(sess, model, operations)
logits, modelParams, modelInfo = model
for layerName in modelInfo:
    variables = modelInfo[layerName]['variables']
    print(variables)

In [ ]:
%%HTML

<style>
#pred_table{
    padding: 3;
    margin: 0;    
    width: 100%;
}

#pred_table table{
    border: none;
    width: 100%;
}

#pred_table div, #pred_table table{
    padding: 0;
    margin: 0;
}

#pred_table table tr td:nth-child(1){
    width: 100px;
}

#pred_table table tr td:nth-child(2){
    width: 40px;
}

#pred_table table tr td:nth-child(3){
    width: 300px;
}

.prob_bar {
    background-color:red;
    height:20px;
}

.prob_bar:first-child {
    background-color:darkgreed;
}

.prob_float{
    position:relative;
}

</style>

In [15]:
import cv2, glob, os.path
import tensorflow as tf
from collections import OrderedDict
from operator import itemgetter 
from htmltoolkit import HtmlToolkit 


def optimistic_restore(session, save_file, graph=tf.get_default_graph(), blacklist = []):
    reader = tf.train.NewCheckpointReader(save_file)
    saved_shapes = reader.get_variable_to_shape_map()
    var_names = sorted([(var.name, var.name.split(':')[0]) for var in tf.global_variables()
            if var.name.split(':')[0] in saved_shapes])    
    restore_vars = []
    #skip_names = []
    #restore_names = []
    for var_name, saved_var_name in var_names:   
        valid = True
        for patterns in blacklist:
            #skip_names.append(var_name) 
            valid = valid and saved_var_name.find(patterns) == -1
            
        if valid:
            curr_var = graph.get_tensor_by_name(var_name)
            var_shape = curr_var.get_shape().as_list()
            if var_shape == saved_shapes[saved_var_name]:
                #restore_names.append( saved_var_name)
                restore_vars.append(curr_var)
            else:
                pass
                #skip_names.append(saved_var_name) 
    #print('Skipped:\n',skip_names)
    #print('Restored: \n',restore_names)
    #print('Restored vars:\n',restore_vars)
    opt_saver = tf.train.Saver(restore_vars, allow_empty=True)
    opt_saver.restore(session, save_file)
    


dataset_path = './dataset/new-traffic-sign/'
htk = HtmlToolkit()
js,jQuery = htk.js, htk.jQuery

X_new = []
y_new = []
for path in glob.glob(dataset_path+'*.jpg'):
    X_new.append(cv2.imread(path,cv2.IMREAD_COLOR))
    # exctract the label number from the full path '<some-dir>/<img-num>_<label>.jpg' 
    y_new.append(int(os.path.basename(path).split('.')[0].split('_')[1])) 
n_new = len(X_new)


rows = [[{'content':'', 'attr':{'colspan':2}},'Label','Desc','Match','Predictions']]
for idx in range(n_new):
    num = str(idx)
    img = X_new[idx]
    label = y_new[idx]
    label_num = str(label)
    text = class_texts[label]
    cols = [
        num,
        htk.embedImageArray(img,{'id':'img_'+num}),
        htk.buildTag('div',{'id':'label_'+num},label_num),
        htk.buildTag('div',{'id':'text_'+num},text),
        htk.buildTag('div',{'id':'match_'+num},'-'),
        htk.buildTag('div',{'id':'pred_'+num},'-')
    ]
    rows.append(cols)

htk.printHTML(htk.htmlTable(rows,{'id':'pred_table'}))
X_new = [processImage(image) for image in X_new]
X_new = np.array(X_new)

basedir = './checkpoints/'
restore_name ='20170326-090427' #load old model 
model_name_load = basedir + restore_name+'/model-'+restore_name
check_name_load = basedir + restore_name+'/'
info = utils.loadModelInfo(basedir,restore_name)
hyperparams = info['hyperparams']
#print(hyperparams)

blacklist = ['Adam','_power'];

with tf.variable_scope("lallo",reuse=True):
    with tf.Session() as sess:
            #load weight & baias from the disk
        sess.run(tf.global_variables_initializer())


        x = tf.placeholder(tf.float32, (None, 32, 32, 3))
        y = tf.placeholder(tf.int32, (None))
        one_hot_y = tf.one_hot(y, n_classes)

        logits, modelParams, modelInfo = BuildNet(x,hyperparams)
        model = tf.nn.softmax(logits)
        #cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)

        feed_dict = {}
        for layerName in modelInfo:
            layerInfo = modelInfo[layerName]
            for name in layerInfo['placeholders']:
                if name.endswith('_keep_prob'):
                    plc = layerInfo['placeholders'][name]
                    layerParam = modelParams[layerName]
                    feed_dict[plc] = 1.0
                else:
                    print('Trovato placeholder non gestito :(',name)



        optimistic_restore(sess, model_name_load,blacklist=blacklist)

        #t = modelInfo['0_conv']['variables']['0_conv_W']
        #print(t.name)#,t.eval())
        #tens = tf.get_default_graph().get_tensor_by_name('0_conv_W')
        #print(tens.eval())

        max_bar_length = 100
        predictions = {}
        num_pred = 5
        print(len(X_new))
        for i in range(n_new):
            num = str(i)
            feed_dict.update({ x: X_new[i:i+1], y: y_new[i:i+1] })
            #feed_dict.update({ x: X_new[0:1], y: y_new[0:1] })
            prob = sess.run(model, feed_dict=feed_dict)[0]
            prob_list = []
            for jk in range(len(prob)):
                prob_list.append({
                    'prob':prob[jk],
                    'label': jk
                });
            #print(prob_list)
            prob_list_sorted = sorted(prob_list, key=lambda x: x['prob'], reverse=True)
            #print(prob_list_sorted)

            top5 = prob_list_sorted[:num_pred]
            top5rows = []
            for item in top5:
                prob,label = item['prob'],item['label']
                text = class_texts[label]
                #print(prob,label, text)
                top5rows.append([
                    htk.buildTag('div', { 'class':'prob_bar', 'style':'width: {:.0f}%'.format(prob*max_bar_length) } ) +
                    htk.buildTag('div', {'class':'prob_float'}, "{:.0f}%".format(prob*100) ),
                    str(label),
                    text,
                ])
            table = htk.htmlTable(top5rows, {'id':'pred_table_'+num})
            #print(table)
            print(num)
            jQuery('#pred_'+num).html(table)
print("mmm")


input      :	 (?, 32, 32, 3) 	 3072
0_conv     :	 (?, 32, 32, 10) 	 10240
1_conv     :	 (?, 32, 32, 20) 	 20480
2_pool     :	 (?, 16, 16, 20) 	 5120
3_conv     :	 (?, 14, 14, 40) 	 7840
4_conv     :	 (?, 12, 12, 60) 	 8640
5_pool     :	 (?, 6, 6, 60) 	 2160
6_conv     :	 (?, 4, 4, 80) 	 1280
7_flat     :	 (?, 1280) 	 1280
8_fc       :	 (?, 600) 	 600
9_fc       :	 (?, 400) 	 400
10_fc      :	 (?, 200) 	 200
11_fc      :	 (?, 100) 	 100
12_fc      :	 (?, 43) 	 43
10


FailedPreconditionError: Attempting to use uninitialized value lallo/0_conv_W
	 [[Node: lallo/0_conv_W/read = Identity[T=DT_FLOAT, _class=["loc:@lallo/0_conv_W"], _device="/job:localhost/replica:0/task:0/gpu:0"](lallo/0_conv_W)]]
	 [[Node: lallo/Softmax/_3 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_12_lallo/Softmax", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'lallo/0_conv_W/read', defined at:
  File "/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

---

## Step 4: Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [12]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it maybe having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

### Question 9

Discuss how you used the visual output of your trained network's feature maps to show that it had learned to look for interesting characteristics in traffic sign images


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 